In [ ]:
import requests
import json

from apikeys import apikey_coinio as key

import sqlite3
from sqlite3 import Error

import glob
import pandas as pd

from sqlalchemy import create_engine

In [ ]:
def initialise_db():
    database = r"crypto_db.sqlite"
    
    sql_create_assets_table = """ CREATE TABLE IF NOT EXISTS "assets" (
        "asset_id" VARCHAR   NOT NULL,
        "name" VARCHAR   NOT NULL,
        "type_is_crypto" INT   NOT NULL,
        "data_quote_start" VARCHAR   NOT NULL,
        "data_quote_end" VARCHAR   NOT NULL,
        "data_orderbook_start" VARCHAR   NOT NULL,
        "data_orderbook_end" VARCHAR   NOT NULL,
        "data_trade_start" VARCHAR   NOT NULL,
        "data_trade_end" VARCHAR   NOT NULL,
        "data_quote_count" VARCHAR   NOT NULL,
        "data_trade_count" VARCHAR   NOT NULL,
        "data_symbols_count" INT   NOT NULL,
        "volume_1hrs_usd" FLOAT   NOT NULL,
        "volume_1day_usd" FLOAT   NOT NULL,
        "volume_1mth_usd" FLOAT   NOT NULL,
        "price_usd" FLOAT   NOT NULL,
        PRIMARY KEY ("asset_id"),
        FOREIGN KEY ("asset_id") REFERENCES "historic_trades" ("asset_id")
    );"""

    sql_create_periods_table = """ CREATE TABLE IF NOT EXISTS "periods" (
        "period_id" VARCHAR   NOT NULL,
        "length_seconds" INT   NOT NULL,
        "length_months" INT   NOT NULL,
        "unit_count" INT   NOT NULL,
        "unit_name" VARCHAR   NOT NULL,
        "display_name" VARCHAR   NOT NULL
    );"""

    sql_create_current_rates_table = """ CREATE TABLE IF NOT EXISTS "current_rates" (
        "time" VARCHAR   NOT NULL,
        "asset_id_base" VARCHAR   NOT NULL,
        "asset_id_quote" VARCHAR   NOT NULL,
        "rate" FLOAT   NOT NULL,
        FOREIGN KEY ("asset_id_base") REFERENCES "assets" ("asset_id")
    );"""

    sql_create_exchanges_table = """ CREATE TABLE IF NOT EXISTS "exchanges" (
        "exchange_id" VARCHAR   NOT NULL,
        "website" VARCHAR   NOT NULL,
        "name" VARCHAR   NOT NULL,
        "data_start" VARCHAR   NOT NULL,
        "data_end" VARCHAR   NOT NULL,
        "data_quote_start" VARCHAR   NOT NULL,
        "data_quote_end" VARCHAR   NOT NULL,
        "data_orderbook_start" VARCHAR   NOT NULL,
        "data_orderbook_end" VARCHAR   NOT NULL,
        "data_trade_start" VARCHAR   NOT NULL,
        "data_trade_end" VARCHAR   NOT NULL,
        "data_symbols_count" INT   NOT NULL,
        "volume_1hrs_usd" FLOAT   NOT NULL,
        "volume_1day_usd" FLOAT   NOT NULL,
        "volume_1mth_usd" FLOAT   NOT NULL
    );"""

    sql_create_historic_trades_table = """ CREATE TABLE IF NOT EXISTS "historic_trades" (
        "asset_id" VARCHAR  NOT NULL,
        "time_period_start" VARCHAR   NOT NULL,
        "time_period_end" VARCHAR   NOT NULL,
        "time_open" VARCHAR   NULL,
        "time_close" VARCHAR   NULL,
        "price_open" FLOAT   NOT NULL,
        "price_high" FLOAT   NOT NULL,
        "price_low" FLOAT   NOT NULL,
        "price_close" FLOAT   NOT NULL,
        "volume_traded" FLOAT   NOT NULL,
        "trades_count" INT   NULL
    );"""
    
    def create_connection(db_file):
        """ create a database connection to the SQLite database specified by db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)
        return conn
    # END create_connection

    def execute_sql_cmd(conn, command):
        """ run a sql command statement
        :param conn: Connection object
        :param execute_sql_cmd: run sql statement
        :return:
        """
        try:
            c = conn.cursor()
            c.execute(command)
        except Error as e:
            print(e)
    # END execute_sql_cmd

    conn = create_connection(database)
    
    if conn is not None:
        execute_sql_cmd(conn, sql_create_assets_table)
        execute_sql_cmd(conn, sql_create_periods_table)
        execute_sql_cmd(conn, sql_create_current_rates_table)
        execute_sql_cmd(conn, sql_create_exchanges_table)
        execute_sql_cmd(conn, sql_create_historic_trades_table)

    else:
        print("Error! cannot create the database connection.")
# END initialise_db


def data_get():
    find_csvs = glob.glob("data_raw/*.csv")

    for i in find_csvs:
        df_csv = pd.read_csv(i)
        df_csv['asset_id'] = find_csvs[find_csvs.index(i)].split("\\", 1)[1].split("_", 2)[0]
        df_csv.dropna(inplace = True)
        df_csv = df_csv.drop(columns=['Adj Close'])
        df_csv = df_csv.rename(columns ={
            'Date' : 'time_period_end',
            'Open' : 'price_open',
            'High' : 'price_high',
            'Low' : 'price_low',
            'Close' : 'price_close',
            'Volume' : 'volume_traded',
        })
        df_csv["time_period_end"] = pd.to_datetime(df_csv["time_period_end"],format='%d/%m/%Y')

#         with open(f'data_raw/{df_csv.asset_id[0]}.json', 'w') as f:
#             f.write(df_csv.to_json())
#         with open( f'data_raw/{df_csv.asset_id[0]}.json', 'r' ) as jj:
#             json_d = json.load( jj )
#             df = pd.DataFrame( json_d )
#         display(df)
# END data_get














def data_clean():
    pass
# END data_clean

def data_update():
    pass
# END data_update

initialise_db()
#data_get()

In [ ]:
find_csvs = glob.glob("data_raw/*.csv")

for i in find_csvs:
    df_csv = pd.read_csv(i)
    df_csv['asset_id'] = find_csvs[find_csvs.index(i)].split("\\", 1)[1].split("_", 2)[0]
    #df_csv['Date'] = df_csv['Date'].str.replace('/', '-')
    #YMD
    df_csv.dropna(inplace = True)
    df_csv = df_csv.drop(columns=['Adj Close'])
    df_csv = df_csv.rename(columns ={
        'Date' : 'time_period_end',
        'Open' : 'price_open',
        'High' : 'price_high',
        'Low' : 'price_low',
        'Close' : 'price_close',
        'Volume' : 'volume_traded',
    })
    
    df_csv['time_period_end'] = pd.to_datetime(df_csv['time_period_end'],format='%d/%m/%Y')
#     df_csv['time_period_start'] = df_csv['time_period_end'].apply(pd.DateOffset(-1))
    df_csv['time_period_start'] = df_csv['time_period_end'].copy()
    
    display(df_csv)
    
    with open(f'data_raw/{df_csv.asset_id[0]}.json', 'w') as f:
        f.write(df_csv.to_json())
    
    con = sqlite3.connect("crypto_db.sqlite")

    df_csv.to_sql("historic_trades", con, if_exists="replace", index=False)

    con.close()


In [ ]:
df_csv.info()

In [ ]:
con = sqlite3.connect("crypto_db.sqlite")

aa = pd.read_sql_query("SELECT * from historic_trades WHERE asset_id='brentoil'", con)
bb = pd.read_sql_query("SELECT * from historic_trades WHERE asset_id='goldfutures'", con)
# df_csv.to_sql("historic_trades", con, if_exists="replace")

con.close()

display(aa, bb)